# Phylogeny-Informed MAG Assessment (PIMA)

Phylogeny-informed MAG assessment (PIMA) is a automatic tool to assess the quality of giant virus MAGs. 

This approach was designed to overcome the limitations caused by a lack of reference genomes, which impacts the accuracy of quality assessments for giant virus MAGs. This approach requires a guide tree of giant viruses that has been rerooted in accordance with the latest taxonomic classifications and evolutionary scenarios. Then, the relative evolutionary divergence (RED) values were calculated to classify taxonomic levels for each clade (e.g., order and family)72. Within a specific clade, MAG genes were annotated with orthologous groups (OGs); then, core genes in this clade were defined as those identified in more than 50% of the genomes in the clade.

Redundant genes in a MAG are defined as genes with more copies than the mode copy number (the most common number of copies) for the given genes across all MAGs in the evaluated lineage.

Citation:

"Genome-resolved year-round dynamics reveal a broad range of giant virus microdiveristy" In Prep.

In [14]:
# load all necessary packagesfor PIMA

from ete3 import Tree
import ete3
from Bio import SeqIO
import subprocess
import os, sys
import pandas as pd
import numpy as np

os.system("mkdir -p tmp")
os.system("mkdir -p output")

0

In [4]:
# Input the phylogenetic tree and reroot it. Here, I used the same dataset (1065 MAGs + reference genomes) from our cited reference paper as the test dataset.
# The input tree was made by a sequence of concatenated marker genes in informational module.
treefile = "test_input.treefile"
tree = Tree(f"{treefile}", quoted_node_names=True, format=1)

# You need to reroot the tree based on the knowlege for the coming RED calculation.
ancestor = tree.get_common_ancestor("165.Flamingopox.virus.FGPVKD09.proteins","073.African.swine.fever.virus.proteins")
tree.set_outgroup(ancestor)
# tree.write(format=1, outfile=f"rerooted_{treefile}")

nodeNb=0
tree.name = f"INT_{nodeNb}"  # Assign INT_0 to the root explicitly
for node in tree.iter_descendants("preorder"):
        #print node.support
        nodeNb+=1
        #print nodeNb
        if not node.is_leaf():
                node.name = "INT_%d" % nodeNb #If I do this only I loose node support..
                #node.add_feature("label", "INT_%d" % nodeNb)
                #node.add_feature("confidence", "%f" % node.support)
                
    
tree.write(format=1, outfile=f"tmp/renamed_rerooted_{treefile}")

In [8]:
# Traverse all nodes to obtain the vertical information between nodes for grouping.

outfile = open("tmp/parent_children.tsv","w")
for node in tree.traverse("postorder"):
    outfile.write(f"\n{node.name}")
    for node2 in (tree&f"{node.name}").get_children():
        outfile.write(f"\t{node2.name}")
        
outfile.close()

# Calculate the RED value
rscript_path = "/usr/local/bin/Rscript"
r_script = "RED_script.R"

result = subprocess.run([rscript_path, r_script], capture_output=True, text=True)

if result.returncode == 0:
    print("R script ran successfully.")
    print(result.stdout)
else:
    print("Error occurred:")
    print(result.stderr)

R script ran successfully.



In [10]:
# This cell is for using the RED value to group genomes
with open("tmp/RED.tsv","r") as infile:
    head = infile.readline()
    lines = infile.readlines()
    reddict = {}
    for line in lines:
        tmp = line.strip().split("\t")
        if len(tmp) > 1:
            reddict[tmp[0]] = tmp[1] 

# I used a RED value of 0.65 as the cut-off, which represents the taxanomy at family level
with open("tmp/parent_children.tsv","r") as infile:
    lines = infile.readlines()
    family = []
    for line in lines:
        tmp = line.strip().split("\t")
        if len(tmp) == 3:
            if float(reddict[tmp[0]]) < 0.65 and float(reddict[tmp[1]]) > 0.65 and float(reddict[tmp[2]]) > 0.65:
                family.append(tmp[0])
            elif float(reddict[tmp[0]]) < 0.65 and float(reddict[tmp[1]]) < 0.65 and float(reddict[tmp[2]]) > 0.65:
                family.append(tmp[2])
            elif float(reddict[tmp[0]]) < 0.65 and float(reddict[tmp[1]]) > 0.65 and float(reddict[tmp[2]]) < 0.65:
                family.append(tmp[1])
                    
family = list(set(family))
print(len(family))


tree = Tree(f"tmp/RED_renamed_rerooted_{treefile}", quoted_node_names=True, format=1)

outfile = open("tmp/family_assignation.tsv","w")

i = 1
for node in family:
    n = 0
    outfile.write(f"\n{node}")
    tmp = []
    for node2 in (tree&f"{node}").get_leaves():
        n += 1
        i+=1
        tmp.append(node2.name)
    outfile.write(f"\t{n}\t{tmp}")
print(i)      
outfile.close()

117
1273


In [11]:
# Then we need to annotation the genes with OG lavels, before that we modify the id to make the gene and genomes easy to match
path = "test_faa/"
dirs = os.listdir( path )
os.system("mkdir -p renamed_original_faa")

i = 1
idrecord = open("tmp/genome_name_record.txt","w")
for file in dirs:
    if ".faa" in file:
        genome = f"genome{i}"
        idrecord.write(f"\n{file}\t{genome}")
        original_file = f"{path}/{file}"
        corrected_file = f"renamed_original_faa/{file}"
        with open(original_file) as original:
            corrected = open(corrected_file, 'w')
            records = SeqIO.parse(original_file, 'fasta')
            n = 1
            for record in records:       
                record.id = f"{genome}_gene{n}"
                n+=1
                SeqIO.write(record, corrected, 'fasta')
        i += 1
    
idrecord.close()
os.system("cat renamed_original_faa/*.faa > all_protein.faa")
os.system("rm -rf renamed_original_faa")


0

In [64]:
# The running cost of hmmsearh could be huge, so I suggested to run this script out of this jupyter notebook
# The hmm file of GVOG DB is availble via https://faylward.github.io/GVDB/. Thank you Frank!
os.system("hmmsearch --cpu 8 --notextw -E 1e-5 --tblout GVOG.out GVOG_hmm/gvog.complete.hmm all_protein.faa > /dev/null 2>&1")

0

In [2]:
# Some steps to keep the best hit of each gene. This might be improved in future.
input_file = "tmp/GVOG.out"
output_file = "tmp/GVOG_best_hits.out"

data = []

# Parse the GVOG.out file
with open(input_file, 'r') as f:
    for line in f:
        if not line.startswith("#"):
            # Split line into parts, keeping the description field as a single column
            parts = line.strip().split(maxsplit=18)
            data.append(parts)

columns = [
    "gene", "target_name", "query_name", "accession", "e_value", "score", "bias",
    "domain_e_value", "domain_score", "domain_bias", "exp", "reg", "clu", "ov",
    "env", "dom", "rep", "inc", "description"
]

data_df = pd.DataFrame(data, columns=columns)

numeric_cols = ["e_value", "score", "domain_e_value", "domain_score", "bias", "domain_bias"]
for col in numeric_cols:
    data_df[col] = pd.to_numeric(data_df[col], errors='coerce')
best_hits = data_df.sort_values(by=["gene", "e_value", "score"], ascending=[True, True, False])
best_hits = best_hits.drop_duplicates(subset="gene", keep="first")

best_hits.to_csv(output_file, sep="\t", index=False)

Best hits for each gene have been saved to GVOG_best_hits.out


In [15]:
# Prepare a master table for the completeness and redundancy calculation

with open("tmp/GVOG_best_hits.out","r") as infile:
    head = infile.readline()
    lines = infile.readlines()
    gvogdict = {}
    for line in lines:
        tmp = line.strip().split()
        genome = tmp[0].split("_")[0]
        for i in tmp[0].split("_")[1:-1]:
            genome += i
        if tmp[2] not in gvogdict.keys():
            gvogdict[tmp[2]] = [1,[genome]]
        else:
            gvogdict[tmp[2]][0] += 1
            gvogdict[tmp[2]][1].append(genome)
    
    obslist = []
    for key in gvogdict.keys():
        if gvogdict[key][0] >= 3:
           obslist.append(key)
print(len(obslist))

genomelist = []
with open("tmp/genome_name_record.txt","r") as infile:
    head = infile.readline()
    lines = infile.readlines()
    for line in lines:
        tmp = line.strip().split("\t")
        genomelist.append(tmp[1])
        
print(len(genomelist))

zeromatrix = np.zeros((len(genomelist),len(obslist)))
df = pd.DataFrame(zeromatrix, index = genomelist,columns = obslist)
for key in gvogdict.keys():
    if key in obslist:
        for origin in gvogdict[key][1]:
            df[key][origin] += 1
df.to_csv("tmp/gvog_genome_master_table.tsv", sep='\t' )

7252
1273


/var/folders/h_/wz31y_q548q_8dx_n56kg2hw0000gn/T/ipykernel_86160/3250747070.py:39: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df[key][origin] += 1


In [18]:
# Prepare the information for final output
namedict = {}
with open("tmp/genome_name_record.txt","r") as infile:
    head = infile.readline()
    lines = infile.readlines()
    for line in lines:
        tmp = line.strip().split("\t")
        namedict[tmp[0].replace("_",".").replace(".faa","")] = tmp[1]
        
df = pd.read_table('tmp/gvog_genome_master_table.tsv', index_col=0)

with open("tmp/gvog_genome_master_table.tsv","r") as infile:
    head = infile.readline()
    GVOGlist = head.strip().split("\t")

In [21]:
# output the clade, at RED of 0.65, information

outfile = open("output/Information_Clades.tsv","w")
outfile.write("Clade\tNumber_Coregene\tCoregene")
with open("tmp/family_assignation.tsv","r") as infile:
    head = infile.readline()
    lines = infile.readlines()
    gvogdict = {}
    for line in lines:
        tmp = line.strip().split("\t")
        if int(tmp[1]) > 2:
            outfile.write(f"\n{tmp[0]}")
            coreset = []
            for govg in GVOGlist:
                i = 0
                for name in eval(tmp[2]):
                    if df[govg][namedict[name]] > 0:
                        i += 1
                # 50% as core gene set
                if i > 0.5 * int(tmp[1]):
                    coreset.append(govg)
            gvogdict[tmp[0]] =coreset
            outfile.write(f"\t{len(coreset)}\t{coreset}")
                    
outfile.close() 

In [20]:
# output the quality of individual MAGs.
outfile = open("output/Quality_MAGs.tsv","w")
outfile.write("MAG\ttaxa\tcompleteness\tcontamination")
with open("tmp/family_assignation.tsv","r") as infile:
    head = infile.readline()
    lines = infile.readlines()
    for line in lines:
        
        tmp = line.strip().split("\t")
        if int(tmp[1]) > 2:
            # calculate core gene number
            paradict = {}
            if len(gvogdict[tmp[0]]) > 0:
                    for gene in gvogdict[tmp[0]]:
                        paratmp = []
                        for name in eval(tmp[2]):
                            if df[gene][namedict[name]] > 0:
                                paratmp.append(df[gene][namedict[name]])
                        counts = np.bincount(paratmp)
                        paradict[gene] = np.argmax(counts)
                        
            for name in eval(tmp[2]):
                c = 0
                r = 0
                if len(gvogdict[tmp[0]]) > 0:
                    for gene in gvogdict[tmp[0]]:
                        if df[gene][namedict[name]] > 0:
                            c += 1
                            if df[gene][namedict[name]] > paradict[gene]:
                                r += 1
                                
                    complete = c*100/len(gvogdict[tmp[0]])
                    contamination = r*100/len(gvogdict[tmp[0]])
                    outfile.write(f"\n{name}\t{tmp[0]}\t{complete}\t{contamination}")
                else:
                    outfile.write(f"\n{name}\t{tmp[0]}\tNA\tNA")
        else:
            for name in eval(tmp[2]):
                outfile.write(f"\n{name}\t{tmp[0]}\tNA\tNA")
                    
outfile.close()   